In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from collections import defaultdict as dd
from itertools import chain
from collections import namedtuple

import os
os.chdir(r'C:\Users\Michal\Documents\Studia\eksploracja_tekstow')
import pickle
import shelve
import unicodedata

import time
from termcolor import colored

from tqdm import trange, tqdm, tqdm_notebook
from sklearn.neighbors import KNeighborsClassifier

In [2]:
Article = namedtuple("Article", ["title", "header", "content"])

In [ ]:
# stop_words = set()
# for line in open("dane\stop_words.txt", 'r', encoding='utf8'):
#     stop_words.add(line[:-1])
# punctuation = set(["'", "(", ")", "-", "!", "[", "]", "–", "", "&", "?", "''", ":", ';', '``', '.', '``', '„', '”', ','])

In [ ]:
# word2vec = dd(list)
# for line in open(r'dane\wiki-forms-all-100-cbow-hs.txt', 'r', encoding='utf8'):
#     line = line[:-1].split(' ')
#     if len(line) != 101:
#         continue
#     word = line[0].lower()
#     embs = [float(val) for val in line[1:]]
#     word2vec[word] = embs

In [3]:
# if not os.path.exists('.cache'):
#     os.makedirs('.cache')
    
# with open(f'.cache/word2vec.pickle', 'wb') as file:
#     pickle.dump(word2vec, file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'.cache/word2vec.pickle', 'rb') as file:
    word2vec = pickle.load(file)

In [ ]:
# with open(r'dane\fp_wiki.txt', 'r', encoding='utf8') as f:
#     wikipedia_raw = f.read()
# articles_raw = wikipedia_raw.split('\n\n')

# wikipedia_db = []
# for article_raw in tqdm(articles_raw):
#     article_lines = article_raw.split('\n')

#     if len(article_lines) < 2:
#         continue

#     title = article_lines[0][7:]
#     header = article_lines[1]
#     content = '\n'.join(article_lines[2:])
#     if len(content) == 0:
#         continue

#     article = Article(title, header, content)
#     wikipedia_db.append(article)

In [4]:
# if not os.path.exists('.cache'):
#     os.makedirs('.cache')
    
# with open(f'.cache/wikipedia.pickle', 'wb') as file:
#     pickle.dump(wikipedia_db, file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'.cache/wikipedia.pickle', 'rb') as file:
    wikipedia_db = pickle.load(file)

In [5]:
title_list_raw = []
for article in tqdm_notebook(wikipedia_db):
    title_list_raw.append(article.title)
title_list = [x.lower().replace(" ","") for x in title_list_raw]

In [9]:
# short_wiki = []
# for art in tqdm_notebook(wikipedia_db):
#     good_words = []
    
#     title = nltk.word_tokenize(art.title)
#     for word in title:
#         word = word.lower()
#         if word in stop_words or word in punctuation:
#             continue
#         good_words.append(word)
        
#     content = nltk.word_tokenize(art.content)
#     for word in content[:100]:
#         word = word.lower()
#         if word in stop_words or word in punctuation:
#             continue
#         good_words.append(word)
        
#     short_article = (good_words)[:50]
#     short_wiki.append(short_article)

In [6]:
# if not os.path.exists('.cache'):
#     os.makedirs('.cache')
    
# with open(f'.cache/short_wiki.pickle', 'wb') as file:
#     pickle.dump(short_wiki, file, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'.cache/short_wiki.pickle', 'rb') as file:
    short_wiki = pickle.load(file)

In [11]:
# wiki_embs = []
# for i, art in enumerate(tqdm_notebook(short_wiki)):
#     art_emb = np.zeros(100)
#     for word in art:
#         word = word.lower()
#         if word2vec[word] == []:
#             continue
#         emb = np.array(word2vec[word])
#         art_emb += emb
#     wiki_embs.append((i, art_emb))

In [7]:
# if not os.path.exists('.cache'):
#     os.makedirs('.cache')
    
# with open(f'.cache/wiki_embs.pickle', 'wb') as file:
#     pickle.dump(wiki_embs, file, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'.cache/wiki_embs.pickle', 'rb') as file:
    wiki_embs = pickle.load(file)

In [8]:
len(wiki_embs), len(short_wiki), len(title_list)

(1159857, 1159857, 1159857)

In [14]:
k = 10
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit([w[1] for w in wiki_embs], [w[0] for w in wiki_embs])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [10]:
def get_ngrams(sentence, n=2):
    ngram = []
    for it in range(len(sentence) - n + 1):
        ngram.append(sentence[it : it + n])
    return ngram

In [11]:
def title_search(q, bdr=5, n=2):
    query = q.lower().replace(" ","")
    query_len = len(query)
    
    grams = get_ngrams(query, n)
    
    candidates_long = [x for x in title_list if (len(x) >= (query_len - bdr) and len(x) <= (query_len + bdr))]

    final_score = []
    for title in candidates_long:
        title_score = 0
        for gram in grams:
            if gram in title:
                title_score += 1
        final_score.append(title_score)
    
    max_val = final_score[np.argmax(final_score)]
    best_fits = [i for i, x in enumerate(final_score) if x == max_val]
    
    if len(best_fits) > 1:
        best_titles =  sorted([(candidates_long[i], i) for i in best_fits], key=lambda x: len(x[0]))
        best_trimmed = [(title[: title.find('(')], i) for title, i in best_titles]
        
        init_len = len(best_trimmed[0][0])
        i = 1
        while i < len(best_trimmed) and len(best_trimmed[i][0]) == init_len:
            i += 1
        
        if i > 1:
            best_earliest = sorted(best_trimmed[:i], key=lambda x: x[1])
            best_fits = [best_earliest[0][1]]
            
            j = 1
            while j < i and best_earliest[j][0] == best_earliest[j-1][0]:
                j += 1
            i = j
            
        best_fits = [i for _, i in best_trimmed[:i]]
    best_fits = [title_list.index(np.array(candidates_long)[i]) for i in best_fits]
    
    return best_fits

In [12]:
def find_similar(query, n_display=5):
    i = title_search(query)[0]
    emb = wiki_embs[i][1]
    
    print(f'QUERY: {query}\n')
    for i in knn.kneighbors(emb.reshape(1,-1), return_distance=False)[0][:n_display]:
        art = wikipedia_db[i]
        print(art.title + '\n')
        print(art.content + '\n')
        print('-' * 100 + '\n')

In [15]:
find_similar('właasadcapierccienai', 10)

QUERY: właasadcapierccienai

Władca Pierścieni

Władca Pierścieni ( ang . `` The Lord of the Rings '' ) – powieść `` high fantasy '' J.R.R . Tolkiena , której akcja rozgrywa się w mitologicznym świecie Śródziemia .
Jest ona kontynuacją innej powieści tego autora zatytułowanej `` Hobbit , czyli tam i z powrotem '' .
`` Władca Pierścieni '' jest często mylnie nazywany trylogią , choć w rzeczywistości jest to powieściowa całość , podzielona na sześć ksiąg . Ze względu na swoją dużą objętość została wydana w trzech tomach z powodu nacisków wydawcy ; nie było to jednak oryginalne zamierzenie Tolkiena . Powieść składa się z sześciu części:

----------------------------------------------------------------------------------------------------

Dwie wieże

Dwie wieże ( ang . `` The Two Towers '' ) – drugi tom powieści ( błędnie nazywanej trylogią ) pt . `` Władca Pierścieni '' autorstwa J.R.R . Tolkiena . Tom ten obejmuje księgę trzecią ( `` Zdrada Isengardu '' ) i czwartą ( `` Podróż do Mordoru